In [1]:
# https://nvd.nist.gov/developers/vulnerabilities
import requests
import sqlite3
import threading
import json

PATH="data/WiGLE_Backup.db"

header = {"Authorization": "token user:679c2955085b46e48155b84f4c878844",}

#url="https://services.nvd.nist.gov/rest/json/cves/2.0?virtualMatchString=cpe:2.3:*:xerox:*:*:*:*:*"
URL_keyword_keywordExactMatch="https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch={}&keywordExactMatch"

#url="http://cve.circl.lu/api/browse/XEROX"

cve0="cve_basic"
cve1="cve_metrics_2"
cve2="cve_metrics_3"
cve3="cve_weaknesses"

```
import requests

url = "https://www.opencve.io/api/vendors/lg/cve"
auth = ('YingliDuan', 'ocveLXRdyl844')

response = requests.get(url, auth=auth)

if response.status_code == 200:
    # Success! Do something with the response data
    data = response.json()
    print(len(data))
    print(data)
else:
    # Error
    print(f"Request failed with status code {response.status_code}")
```

```
con = sqlite3.connect(PATH)
cur = con.cursor()
databases=["cve_basic","cve_metrics_2","cve_weaknesses","cve_metrics_3"]
for d in databases:
    sql="DELETE FROM {}".format(d)
    cur.execute(sql)
con.commit()
con.close()
```

In [2]:
import time
def insertDB(theme,results,keywords,extract=False):
    con = sqlite3.connect(PATH)
    c = con.cursor()
    print("----insert vulnerabilities into database----")
    print("******** {},{}********".format(theme,keywords))
    data_es=results['vulnerabilities']
    #print(data_es)
    i_b = 0
    i_m=0
    i_w=0
    for data in data_es:
        for des in data.get('cve').get("descriptions"):
            i_b+=1
            if extract==True:
                keywords=data.get('cve').get("configurations")[0].get('nodes')[0].get('cpeMatch')[0].get('criteria').split(':')[3]
            print(keywords)
            #print(des)
            c.execute('''INSERT OR IGNORE INTO cve_basic (id, published, lastModified, vulnStatus, description_value,keywords)
                 VALUES (?, ?, ?, ?, ?, ?)''',
                 (data.get('cve').get('id'),
                  data.get('cve').get('published'),
                  data.get('cve').get('lastModified'),
                  data.get('cve').get('vulnStatus'),
                  des.get('value'),
                  keywords,
                 ))
        
        cvsses=list(data.get('cve').get("metrics").keys())
        if cvsses is not None:
            for cvss in cvsses:
                items = data.get('cve').get('metrics').get(cvss)
                if "cvssMetricV2" in cvss:
                    for item in items:
                        i_m+=1
                        c.execute('''INSERT OR IGNORE INTO cve_metrics_2 (id, cvss_type,type, cvssVersion, vectorString, accessVector, accessComplexity,
                             authentication, confidentialityImpact, integrityImpact, availabilityImpact, baseScore, baseSeverity,
                             exploitabilityScore, impactScore, acInsufInfo, obtainAllPrivilege, obtainUserPrivilege, obtainOtherPrivilege,
                             userInteractionRequired)
                             VALUES (?, ?, ?, ?,  ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?)''',
                             (data.get('cve').get('id'),
                              cvss,
                              item.get('type'),
                              item.get('cvssData').get('version'),
                              item.get('cvssData').get('vectorString'),
                              item.get('cvssData').get('accessVector'),
                              item.get('cvssData').get('accessComplexity'),
                              item.get('cvssData').get('authentication'),
                              item.get('cvssData').get('confidentialityImpact'),
                              item.get('cvssData').get('integrityImpact'),
                              item.get('cvssData').get('availabilityImpact'),
                              item.get('cvssData').get('baseScore'),
                              item.get('baseSeverity'),
                              item.get('exploitabilityScore'),
                              item.get('impactScore'),
                              item.get('acInsufInfo'),
                              item.get('obtainAllPrivilege'),
                              item.get('obtainUserPrivilege'),
                              item.get('obtainOtherPrivilege'),
                              item.get('userInteractionRequired')))
                if "cvssMetricV3" in cvss:
                    for item in items:
                        i_m+=1
                        c.execute('''INSERT OR IGNORE INTO cve_metrics_3 (id, cvss_type, type, cvssVersion, vectorString, attackVector, attackComplexity,
                                 privilegesRequired, userInteraction, scope, confidentialityImpact, integrityImpact, availabilityImpact, baseScore, baseSeverity,
                                 exploitabilityScore, impactScore)
                                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                                 (data.get('cve').get('id'),
                                  cvss,
                                  item.get('type'),
                                  item.get('cvssData').get('version'),
                                  item.get('cvssData').get('vectorString'),
                                  item.get('cvssData').get('attackVector'),
                                  item.get('cvssData').get('attackComplexity'),
                                  item.get('cvssData').get('privilegesRequired'),
                                  item.get('cvssData').get('userInteraction'),
                                  item.get('cvssData').get('scope'),
                                  item.get('cvssData').get('confidentialityImpact'),
                                  item.get('cvssData').get('integrityImpact'),
                                  item.get('cvssData').get('availabilityImpact'),
                                  item.get('cvssData').get('baseScore'),
                                  item.get('cvssData').get('baseSeverity'),
                                  item.get('exploitabilityScore'),
                                  item.get('impactScore')))
        weaknesses=data.get('cve').get('weaknesses')
        if weaknesses is not None:
            for weak in weaknesses:
                for value in weak['description']:
                    i_w+=1
                    #(data.get('cve').get('id'),weak.get('type'),value)
                    #print(data.get('cve').get('id'),weak.get('type'),value)

                    c.execute('''INSERT OR IGNORE INTO cve_weaknesses (id,type,description_value) VALUES(?,?,?)''',
                            (data.get('cve').get('id'),weak.get('type'),value.get("value")))
    
    con.commit()
    con.close()
    print("cve_basic:", i_b)
    print("cve_metrics:", i_m)
    print("cve_weaknesses:", i_w)
    print("---- DONE {}----".format(keywords))
def collect_load_CVE(types,title,url=None,extract=False):
    results_types=dict.fromkeys(types)
    if url is None:
        url=URL
    threads=[]
    for typ in types:
    #for i in range(213,len(types)):
        #typ=types[i]
        url_n = url.format(typ)
        print(url_n)
        response = requests.get(url_n)
        #print(response.json())
        print(response.status_code)
        time.sleep(6)
        if response.status_code==200:
            print("UPLODING")
            results_types[typ]=response.json()
                
            insertDB(title,results_types[typ],typ,extract)
#             t=threading.Thread(target=insertDB,args=(title,results_types[typ],typ))
#             threads.append(t)
#             t.start()
        #print(response.json())
#     for t in threads:
#         t.join()


# BT_VERSION

In [3]:
collect_load_CVE(["Bluetooth Low Energy"],"Bluetoth Version",url=URL_keyword_keywordExactMatch,extract=True)

https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Bluetooth Low Energy&keywordExactMatch
200
UPLODING
----insert vulnerabilities into database----
******** Bluetoth Version,Bluetooth Low Energy********
360fly
360fly
vaulteksafe
vaulteksafe
vaulteksafe
vaulteksafe
mimobaby
mimobaby
rondaful_project
rondaful_project
o.bike
o.bike
wincofireworks
wincofireworks
mi
mi
google
google
fitbit
fitbit
tapplock
tapplock
tapplock
tapplock
belwith-keeler
belwith-keeler
xiaoyi
xiaoyi
vandyvape
vandyvape
yalehome
yalehome
ti
ti
syska
syska
nxp
nxp
cypress
cypress
dialog-semiconductor
dialog-semiconductor
dialog-semiconductor
dialog-semiconductor
ti
ti
ti
ti
microchip
microchip
telink-semi
telink-semi
telink-semi
telink-semi
cypress
cypress
nxp
nxp
st
st
cypress
cypress
cypress
cypress
google
google
silabs
silabs
silabs
silabs
ti
ti
espressif
espressif
espressif
espressif
google
google
sooil
sooil
sooil
sooil
sooil
sooil
sooil
sooil
sooil
sooil
sooil
sooil
bluetooth
bluetooth
sloan
sloan
b

# Cipher

In [4]:
# {'EAP/SHA1': 3644, 'PSK': 21131, 'PSK+SAE': 176, 'EAP/SHA1+FT/EAP': 298, 'PSK+FT/PSK': 254, 'SAE': 18, 'EAP/SHA1+?': 44, 'PSK+PSK': 9, 'SHA256': 8, 'EAP/SHA1+EAP/SHA256': 6, 'SHA256+SAE': 1}
# ---------cipher-----------
# {'CCMP': 23470, 'TKIP+CCMP': 770, 'CCMP+TKIP': 1211, 'TKIP': 117, 'None+CCMP': 8}
# ---------others-----------
# {'RSN': 12354, 'ESS': 14413, 'WPS': 5857, 'MFPC': 114, 'MFPR': 1, 'IBSS': 2, 'Mis': 2, 'Uncategorized;1': 2}

keywords={'Extensible+Authentication+Protocol':'EAP','Pre-Shared+Key':'PSK','Simultaneous+Authentication+of+Equals':'SAE','CCMP':'CCMP','TKIP':'TKIP','Robust+Security+Network':'RSN','Wi-Fi+Protected+Setup':'WPS'}
collect_load_CVE(keywords.keys(),"WiFi Cipher",url=URL_keyword_keywordExactMatch,extract=True)

https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Extensible+Authentication+Protocol&keywordExactMatch
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Cipher,Extensible+Authentication+Protocol********
cisco
cisco
cisco
cisco
cisco
vocera
vocera
aruba_networks
aruba_networks
microsoft
microsoft
microsoft
microsoft
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
cisco
juniper
juniper
microsoft
microsoft
microsoft
microsoft
microsoft
microsoft
cve_basic: 35
cve_metrics: 31
cve_weaknesses: 30
---- DONE microsoft----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Pre-Shared+Key&keywordExactMatch
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Cipher,Pre-Shared+Key********
sonicwall
safenet
neoteris
microsoft
microsoft
symantec
symantec
ubuntu
ubuntu
hot
hot
strongswan
strongswan
siemens
siemens
cisco
cisco
mimosa
mimosa
ietf
ietf
netgear
netgear
cisco
cisco
moxa
moxa
trendnet
trendnet


## Extract all the company and type and calculate the number

> key words: type("Bluetooth" or "WiFi"), company name

> parameters: time >=2022

In [4]:
ULW_P=["EUROPE","BEIJING","SHENZHEN","AMERICA","SICHUAN","HUI","ZHOU","Hong","KONG","SHANGHAI","ASIA","CH","USA","BELGIUM",
       "ZHUHAI","GUANGZHOU"]# useless word-place
ULW_S=["COMPUTER","SYSTEMS","CO","LTD","LIMITED","CORPORATION","CORP","COMMUNCATIONS","COMMUNICATION","COMMUNICATIONS","ELECTRONIC","ELECTRONICS",
       "NETWORK","INC","LLC","COMPANY","INTERNATIONAL","WIRELESS","DATA","MEDICAL","AG","Electric","NETWORKING","ELECTRIC",
       "SRO","BV","SYS","INDUSTRIALS","INDUSTRIAL","DEVICE","DEVICES","COMM","TECHNOLOGY","TECHNOLOGIES","TECH","COLTD","A/S","DELIVERY","NV",
       "NETWORKS","APPLICATIONS","OF","APS","GROUP","IND","DEVELOPMENT","AS","AB","OY","IOT","INNOTEK","GMBH","AUDIOVISUELLES",
       "MARKETING","UND","COMPUTERSYSTEME","MACHINE","ELECTRO","MECHANICS","MOBILE","SOFTWARE","DIGITAL","TECHLTD","INFORMATION",
      "AUTOMOTIVE","CORPORATE","ENTERPRISE","LABORATORY","LABORATORIES"] # useless word-suffix
def clean_name(vendors):
    vens=[]
    vendors=list(vendors)
    con = sqlite3.connect(PATH)
    cur = con.cursor()
    for vendor in vendors:
        rt=vendor[0].replace(",","").replace(".","").replace("\n","").replace("，","").replace("'","")
        rt_s=rt.split(" ")
        if rt_s[0]=="Harman/Becker":
            name=["Harman"]
        elif rt_s[0]=="Nokia":
            name=["Nokia"]
        else:
            name=[r for r in rt_s if r.upper() not in ULW_P and r.upper() not in ULW_S and "(" not in r and ")" not in r]
        if len(name)!=0:
            sql="select * from {} where keywords='{}'".format(cve0,name[0])
            if len(cur.execute(sql).fetchall())==0:
                vens.append(name[0])
    con.close()
    return vens

In [14]:
con = sqlite3.connect(PATH)
cur = con.cursor()

sql="""SELECT DISTINCT vendor.OrganizationName
        FROM vendor
        JOIN (
            SELECT bssid 
            FROM network 
            WHERE type = 'W'
            UNION ALL 
            SELECT netid 
            FROM wigle_wifi
        ) AS network_wifi ON network_wifi.bssid = vendor.mac
        GROUP BY vendor.OrganizationName
        """
rts=cur.execute(sql).fetchall()
print("******Results of WiFi*******\n",len(rts))
rts=set(clean_name(rts))
#collect_load_CVE(rts,"WiFi vendors")
print("********CLEANED NAME*********\n",len(rts),rts)
con.close()


collect_load_CVE(list(rts),"WiFi Vendors")

******Results of WiFi*******
 285
********CLEANED NAME*********
 268 {'Kaonmedia', 'Winmate', 'eero', 'HMD', 'CC&C', 'EFM', 'ACCTON', 'Siliconware', 'Acer', 'Silicon', 'Agere', 'Global', 'Yunlink', 'TCT', 'Winstars', 'Xiaomi', 'Panasonic', 'EnGenius', 'Yichen', 'GREE', 'Securifi', 'Gaoshengda', 'Philips', 'Fortinet', 'MINIWARE', 'D&M', 'ProCurve', 'SURECOM', 'Tenda', 'Continental', 'AirTies', 'Colubris', 'Delta', '7INOVA', 'Ericsson', 'Nokia', 'Broadcom', 'Slim', 'Chung', 'Xiaoyi', 'Sichuan\xa0AI-Link\xa0Technology\xa0Co\xa0Ltd', 'Brans', 'BMW', 'Creatix', 'Google', 'Ford', 'MitraStar', 'Prime', 'XAVi', 'Gionee', 'InPro', 'u-blox', 'Hewlett', 'HUAWEI', 'BUFFALOINC', 'IEEE', '3D', 'zte', 'Technicolor', 'Allied', 'ESSYS', 'MICRO-STAR', 'Pepwave', 'Compal', 'EWPE', 'Garmin', 'Aruba', 'Actia', 'Voyetra', 'Partron', 'ALPSALPINE', 'Metro-Optix', 'SMC', 'Sagemcom', 'Teltonika', 'Cameo', 'MERCURY', 'Belkin', 'Clipcomm', 'Zyxel', 'Ambit', 'Siemens', 'ASUSTek', 'Bromax', 'Frontier', 'Syabas', 'I

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Fortinet********
cve_basic: 627
cve_metrics: 647
cve_weaknesses: 335
---- DONE Fortinet----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=MINIWARE
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,MINIWARE********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE MINIWARE----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=D&M
404
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=ProCurve
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,ProCurve********
cve_basic: 49
cve_metrics: 25
cve_weaknesses: 25
---- DONE ProCurve----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=SURECOM
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,SURECOM********
cve_basic: 5
cve_metrics: 5
cve_weaknesses: 4
---- DONE SURECOM----
https://services.nvd.nist.gov/rest/json/cve

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Allied********
cve_basic: 33
cve_metrics: 24
cve_weaknesses: 20
---- DONE Allied----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=ESSYS
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,ESSYS********
cve_basic: 4
cve_metrics: 4
cve_weaknesses: 2
---- DONE ESSYS----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=MICRO-STAR
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,MICRO-STAR********
cve_basic: 20
cve_metrics: 16
cve_weaknesses: 11
---- DONE MICRO-STAR----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Pepwave
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Pepwave********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Pepwave----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Compal
200
UPLODING
----insert vulnerabilities into database-

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,DrayTek********
cve_basic: 65
cve_metrics: 71
cve_weaknesses: 36
---- DONE DrayTek----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Dovado
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Dovado********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Dovado----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Routerboardcom
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Routerboardcom********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Routerboardcom----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=TECOM
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,TECOM********
cve_basic: 74
cve_metrics: 51
cve_weaknesses: 40
---- DONE TECOM----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Juan
200
UPLODING
----insert vulnerabilities into 

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,NVIDIA********
cve_basic: 1168
cve_metrics: 1331
cve_weaknesses: 780
---- DONE NVIDIA----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=EQUIPTRANS
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,EQUIPTRANS********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE EQUIPTRANS----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Sur-Gard
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Sur-Gard********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Sur-Gard----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=REALTEK
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,REALTEK********
cve_basic: 107
cve_metrics: 105
cve_weaknesses: 72
---- DONE REALTEK----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=FN-LINK
200
UPLODING
----insert vulnerabiliti

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Hong********
cve_basic: 108
cve_metrics: 110
cve_weaknesses: 61
---- DONE Hong----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Vivotek
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Vivotek********
cve_basic: 50
cve_metrics: 48
cve_weaknesses: 25
---- DONE Vivotek----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Sonicwall
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Sonicwall********
cve_basic: 368
cve_metrics: 346
cve_weaknesses: 303
---- DONE Sonicwall----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Zinwell
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Zinwell********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Zinwell----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=DATABILITY
200
UPLODING
----insert vulnerabilities int

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Luxul********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Luxul----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Huawei
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Huawei********
cve_basic: 3106
cve_metrics: 2942
cve_weaknesses: 1662
---- DONE Huawei----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=AMIT
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,AMIT********
cve_basic: 14
cve_metrics: 8
cve_weaknesses: 8
---- DONE AMIT----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Freecom
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Freecom********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Freecom----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=COMPU-SHACK
200
UPLODING
----insert vulnerabilities into database----
****

ConnectionError: HTTPSConnectionPool(host='services.nvd.nist.gov', port=443): Max retries exceeded with url: /rest/json/cves/2.0?keywordSearch=NEC (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000210590DD430>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [18]:
collect_load_CVE(list(rts),"WiFi Vendors")

https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Seiko
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Seiko********
cve_basic: 20
cve_metrics: 12
cve_weaknesses: 9
---- DONE Seiko----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=NEC
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,NEC********
cve_basic: 3295
cve_metrics: 2003
cve_weaknesses: 2017
---- DONE NEC----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Enterasys
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Enterasys********
cve_basic: 13
cve_metrics: 8
cve_weaknesses: 8
---- DONE Enterasys----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=LG
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,LG********
cve_basic: 3049
cve_metrics: 3100
cve_weaknesses: 1870
---- DONE LG----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Int

200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Nexsi********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Nexsi----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=HCL
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,HCL********
cve_basic: 355
cve_metrics: 388
cve_weaknesses: 249
---- DONE HCL----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=AMPAK
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,AMPAK********
cve_basic: 2
cve_metrics: 1
cve_weaknesses: 1
---- DONE AMPAK----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Bridgeco
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Vendors,Bridgeco********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Bridgeco----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=BILIAN
200
UPLODING
----insert vulnerabilities into database----
******** WiFi Ve

In [16]:
for i in range(len(rts)):
    if list(rts)[i]=='Seiko':
        print(i)
        break

213


In [8]:
con = sqlite3.connect(PATH)
cur = con.cursor()
sql="""SELECT DISTINCT vendor.OrganizationName
        FROM vendor
        JOIN (
            SELECT bssid as netid
            FROM network 
            WHERE type = 'B' or type = 'E'
            UNION ALL 
            SELECT netid,capabilities 
            FROM wigle_blue
        ) AS network_blue
        WHERE network_blue.netid = vendor.mac
        GROUP BY vendor.OrganizationName
        """
rts=cur.execute(sql).fetchall()
#print("******Results of BT*******\n",len(rts),rts)
rts=set(clean_name(rts))
print("********CLEANED NAME*********\n",len(rts),rts)
con.close()
collect_load_CVE(list(rts),"BT Vendors")

********CLEANED NAME*********
 1692 {'', 'COMPUADD', 'SYSTEMFORSCHUNG', 'Avaya', 'Atmel', 'Accu-Sort', 'GOLDSTAR', 'eBet', 'APPRO', 'iVeia', 'DongGuan', 'Indigo', 'Panduit', 'MKNet', 'Sonitor', 'Beats', 'GateWare', 'Rocon', 'NECTEC', 'Multitone', 'VT', 'Innovative', 'KOYO', 'ResMed', 'Nanjing', 'ZEAL', 'CAMTRONICS', 'IPM', 'PRECISION', 'COMPUTERM', 'MediaChorus', 'PJRCCOM', 'OpenRTLS', 'Albentia', 'TIDOMAT', 'VIASAT', 'ORSYS', 'Gassner', 'Huaqin', 'GARO', 'SPECTRIX', 'congatec', 'Audiomatica', 'Holy', 'Isotek', 'DIONEX', '690885', 'iRobot', 'Amercom', 'FONSYS', 'INTEGRATED', 'PHOTON', 'ShinMaywa', '3H', 'Honor', 'Shiyuan', 'Prolon', 'WiSE', 'Value', 'GE', 'Nitgen', 'vivo', 'INNERINT', 'Zinwave', 'Edifier', 'Wuxi', 'SPG', 'Keico', 'Mega-Trend', 'GCI', 'XinHuaTong', 'Wu', 'Middle', 'QTS', 'Atlanta', 'Shenzhenshi', 'Arecont', 'Ambient', 'NGES', 'ARKUS', 'CAI', 'LANACCESS', 'MAT', 'Qingping', 'Telco', 'Olivetti', 'Alcatel', 'KAIMEI', 'EXPRESS', 'HTRC', 'WEST', 'Polypix', 'Sharetronic', 'Br

200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Powerhouse********
cve_basic: 10
cve_metrics: 5
cve_weaknesses: 5
---- DONE Powerhouse----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=WIS
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,WIS********
cve_basic: 1642
cve_metrics: 1580
cve_weaknesses: 1096
---- DONE WIS----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Packet
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Packet********
cve_basic: 3392
cve_metrics: 2001
cve_weaknesses: 2023
---- DONE Packet----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Imagination
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Imagination********
cve_basic: 4
cve_metrics: 4
cve_weaknesses: 2
---- DONE Imagination----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=BRISTOL
200
UPLODING
----insert vulnerabilities into d

200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,MIDDLE********
cve_basic: 3969
cve_metrics: 2004
cve_weaknesses: 2003
---- DONE MIDDLE----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=HUNAN
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,HUNAN********
cve_basic: 6
cve_metrics: 6
cve_weaknesses: 3
---- DONE HUNAN----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Sierra
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Sierra********
cve_basic: 369
cve_metrics: 374
cve_weaknesses: 198
---- DONE Sierra----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=NSM
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,NSM********
cve_basic: 1571
cve_metrics: 1549
cve_weaknesses: 1005
---- DONE NSM----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=G-Star
200
UPLODING
----insert vulnerabilities into database----
******** B

cve_basic: 40
cve_metrics: 21
cve_weaknesses: 21
---- DONE Thai----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Palit
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Palit********
cve_basic: 4
cve_metrics: 4
cve_weaknesses: 2
---- DONE Palit----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Vigil
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Vigil********
cve_basic: 467
cve_metrics: 311
cve_weaknesses: 257
---- DONE Vigil----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=EbV
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,EbV********
cve_basic: 336
cve_metrics: 301
cve_weaknesses: 224
---- DONE EbV----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=CORONA
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,CORONA********
cve_basic: 19
cve_metrics: 16
cve_weaknesses: 12
---- DONE CORONA----
https://serv

200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Any********
cve_basic: 2888
cve_metrics: 1301
cve_weaknesses: 1138
---- DONE Any----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Tellion
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Tellion********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Tellion----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Mayer
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Mayer********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Mayer----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=JNA
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,JNA********
cve_basic: 46
cve_metrics: 45
cve_weaknesses: 35
---- DONE JNA----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Perfisans
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Pe

...

200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Cierma********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Cierma----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=OPENBRAIN
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,OPENBRAIN********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE OPENBRAIN----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=ITTC
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,ITTC********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE ITTC----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Jiangsu
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Jiangsu********
cve_basic: 2
cve_metrics: 2
cve_weaknesses: 1
---- DONE Jiangsu----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=NUCOM
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendo

200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,DigiBoard********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE DigiBoard----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=StreamUnlimited
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,StreamUnlimited********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE StreamUnlimited----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Volktek
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Volktek********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Volktek----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=SymCom
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,SymCom********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE SymCom----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=BACHMANN
200
UPLODING
----insert vulnerabilities in

200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,YVES********
cve_basic: 5
cve_metrics: 4
cve_weaknesses: 3
---- DONE YVES----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=DEFIDEV
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,DEFIDEV********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE DEFIDEV----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=ASCOM
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,ASCOM********
cve_basic: 11
cve_metrics: 11
cve_weaknesses: 6
---- DONE ASCOM----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=VEGA
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,VEGA********
cve_basic: 47
cve_metrics: 47
cve_weaknesses: 32
---- DONE VEGA----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=National
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Natio

cve_basic: 59
cve_metrics: 56
cve_weaknesses: 37
---- DONE Lighting----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=SimpliSafe
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,SimpliSafe********
cve_basic: 14
cve_metrics: 14
cve_weaknesses: 7
---- DONE SimpliSafe----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Amity
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Amity********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Amity----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Azuretec
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Azuretec********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Azuretec----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=NUUO
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,NUUO********
cve_basic: 52
cve_metrics: 52
cve_weaknesses: 36
---- DONE NUU

cve_basic: 3905
cve_metrics: 3638
cve_weaknesses: 2304
---- DONE Virtual----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=YAMASHITA
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,YAMASHITA********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE YAMASHITA----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=xG
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,xG********
cve_basic: 1437
cve_metrics: 1481
cve_weaknesses: 958
---- DONE xG----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Comdial
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Comdial********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Comdial----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Skytone
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Skytone********
cve_basic: 4
cve_metrics: 4
cve_weaknesses: 2
---- DONE Skyt

https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=KeRo
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,KeRo********
cve_basic: 2114
cve_metrics: 2486
cve_weaknesses: 1727
---- DONE KeRo----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Albentia
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,Albentia********
cve_basic: 0
cve_metrics: 0
cve_weaknesses: 0
---- DONE Albentia----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=NBS
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,NBS********
cve_basic: 451
cve_metrics: 349
cve_weaknesses: 280
---- DONE NBS----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=MAPLE
200
UPLODING
----insert vulnerabilities into database----
******** BT Vendors,MAPLE********
cve_basic: 8
cve_metrics: 7
cve_weaknesses: 5
---- DONE MAPLE----
https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch=Infineon
200
U

OperationalError: database is locked

In [31]:
con = sqlite3.connect(PATH)
cur = con.cursor()
con.commit()
con.close()

In [5]:
result_blue['vulnerabilities'][0]

{'cve': {'id': 'CVE-1999-0233',
  'sourceIdentifier': 'cve@mitre.org',
  'published': '1996-02-25T05:00:00.000',
  'lastModified': '2020-11-23T19:49:20.393',
  'vulnStatus': 'Modified',
  'descriptions': [{'lang': 'en',
    'value': 'IIS 1.0 allows users to execute arbitrary commands using .bat or .cmd files.'}],
  'metrics': {'cvssMetricV2': [{'source': 'nvd@nist.gov',
     'type': 'Primary',
     'cvssData': {'version': '2.0',
      'vectorString': 'AV:N/AC:L/Au:N/C:C/I:C/A:C',
      'accessVector': 'NETWORK',
      'accessComplexity': 'LOW',
      'authentication': 'NONE',
      'confidentialityImpact': 'COMPLETE',
      'integrityImpact': 'COMPLETE',
      'availabilityImpact': 'COMPLETE',
      'baseScore': 10.0},
     'baseSeverity': 'HIGH',
     'exploitabilityScore': 10.0,
     'impactScore': 10.0,
     'acInsufInfo': False,
     'obtainAllPrivilege': True,
     'obtainUserPrivilege': False,
     'obtainOtherPrivilege': False,
     'userInteractionRequired': False}]},
  'weakne

## get vulnerabilities according to company name and devices

In [44]:
con.close()

# load them to db

In [ ]:
con=sqlite3.connect(PATH)
c=con.cursor()